In [1]:
# ====================================================
# Directory settings
# ====================================================
import os

OUTPUT_DIR = './Comonlist_024_3'
if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR)

In [2]:
# ====================================================
# Library
# ====================================================
import os
import gc
import re
import ast
import sys
import copy
import json
import time
import math
import string
import pickle
import random
import joblib
import itertools
import warnings
warnings.filterwarnings("ignore")

import scipy as sp
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
from tqdm.auto import tqdm
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import StratifiedKFold, GroupKFold, KFold

os.system('pip install iterative-stratification==0.1.7')
# from iterstrat.ml_stratifiers import MultilabelStratifiedKFold

import torch
import torch.nn as nn
from torch.nn import Parameter
import torch.nn.functional as F
from torch.optim import Adam, SGD, AdamW
from torch.utils.data import DataLoader, Dataset

os.system('pip install -q transformers')
os.system('pip install -q tokenizers')
import tokenizers
import transformers
print(f"tokenizers.__version__: {tokenizers.__version__}")
print(f"transformers.__version__: {transformers.__version__}")
from transformers import AutoTokenizer, AutoModel, AutoConfig
from transformers import get_linear_schedule_with_warmup, get_cosine_schedule_with_warmup
%env TOKENIZERS_PARALLELISM=true

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

tokenizers.__version__: 0.11.6
transformers.__version__: 4.29.2
env: TOKENIZERS_PARALLELISM=true


In [4]:
from torch.cuda.amp import autocast, GradScaler
from sklearn import metrics
from src.machine_learning_util import set_seed, set_device, init_logger, AverageMeter, to_pickle, unpickle, asMinutes, timeSince

In [5]:
class CFG:
    EXP_ID = '024'
    apex = True
    model ='microsoft/deberta-v3-base' #'microsoft/deberta-v3-large' 
    seed = 2022 # 42 # 71
    n_splits = 4
    max_len = 512 # 1429 # 1024 # 512
    dropout = 0
    targets = ['content', 'wording']
    target_size = len(targets)
    n_accumulate=1
    print_freq = 100
    eval_freq = 780 * 2 # 390 # 170
    min_lr=1e-6
    scheduler = 'cosine'
    batch_size = 1 # 2 # 4
    num_workers = 0 #3
    lr = 5e-6 # 3e-6
    weigth_decay = 0.01
    epochs = 4
    n_fold = 4
    trn_fold = [i for i in range(n_fold)]
    train = True
    num_warmup_steps = 0
    num_cycles=0.5
    debug = False
    freezing = True
    gradient_checkpoint = True
    reinit_layers = 4 # 3
    tokenizer = AutoTokenizer.from_pretrained(model)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [6]:
# ====================================================
# Utils
# ====================================================
def MCRMSE(y_trues, y_preds):
    scores = []
    idxes = y_trues.shape[1]
    for i in range(idxes):
        y_true = y_trues[:,i]
        y_pred = y_preds[:,i]
        score = mean_squared_error(y_true, y_pred, squared=False) # RMSE
        scores.append(score)
    mcrmse_score = np.mean(scores)
    return mcrmse_score, scores


def get_score(y_trues, y_preds):
    mcrmse_score, scores = MCRMSE(y_trues, y_preds)
    return mcrmse_score, scores


def get_logger(filename=OUTPUT_DIR+'_train'):
    from logging import getLogger, INFO, StreamHandler, FileHandler, Formatter
    logger = getLogger(__name__)
    logger.setLevel(INFO)
    handler1 = StreamHandler()
    handler1.setFormatter(Formatter("%(message)s"))
    handler2 = FileHandler(filename=f"{filename}.log")
    handler2.setFormatter(Formatter("%(message)s"))
    logger.addHandler(handler1)
    logger.addHandler(handler2)
    return logger

LOGGER = get_logger()


def seed_everything(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    
seed_everything(CFG.seed)
# seed_everything(seed=60)

# Data Loading

In [7]:
# ====================================================
# Data Loading
# ====================================================
path = "C:/Users/Lab000/Desktop/kaggle/kaggle_competetion/CommonLit - Evaluate Student Summaries/input/commonlit-evaluate-student-summaries/"
prompts_train = pd.read_csv(path+'prompts_train.csv')
prompts_test = pd.read_csv(path+'prompts_test.csv')

summaries_train = pd.read_csv(path+'summaries_train.csv')
summaries_test = pd.read_csv(path+'summaries_test.csv')

submission = pd.read_csv(path+'sample_submission.csv')

print(f"prompts_train.shape: {prompts_train.shape}")
#display(prompts_train.head())
print(f"prompts_test.shape: {prompts_test.shape}")
#display(prompts_test.head())

print(f"summaries_train.shape: {summaries_train.shape}")
#display(summaries_train.head())
print(f"summaries_test.shape: {summaries_test.shape}")
#display(summaries_test.head())
print(f"submission.shape: {submission.shape}")
#display(submission.head())

prompts_train.shape: (4, 4)
prompts_test.shape: (2, 4)
summaries_train.shape: (7165, 5)
summaries_test.shape: (4, 3)
submission.shape: (4, 3)


# Merge Prompts Data 

In [8]:
train = summaries_train.merge(prompts_train, how="left", on="prompt_id")
test = summaries_test.merge(prompts_test, how="left", on="prompt_id")

# CV split(GroupKFold)

In [9]:
# ====================================================
# CV split
# ====================================================
Fold = GroupKFold(n_splits=CFG.n_fold)
for n, (train_index, val_index) in enumerate(Fold.split(train, groups=train["prompt_id"])):
    train.loc[val_index, 'fold'] = int(n)
train['fold'] = train['fold'].astype(int)
display(train.groupby('fold').size())

fold
0    2057
1    2009
2    1996
3    1103
dtype: int64

In [10]:
CFG.tokenizer.save_pretrained(OUTPUT_DIR+'/tokenizer/')

('./Comonlist_024_3/tokenizer/tokenizer_config.json',
 './Comonlist_024_3/tokenizer/special_tokens_map.json',
 './Comonlist_024_3/tokenizer/spm.model',
 './Comonlist_024_3/tokenizer/added_tokens.json',
 './Comonlist_024_3/tokenizer/tokenizer.json')

# Text max Length

In [11]:
# ====================================================
# Define max_len
# ====================================================
lengths = []
tk0 = tqdm(train['text'].fillna("").values, total=len(train))
for text in tk0:
    length = len(CFG.tokenizer(text, add_special_tokens=False)['input_ids'])
    lengths.append(length)
CFG.max_len = max(lengths) + 2 # cls & sep
LOGGER.info(f"text column max_len: {CFG.max_len}")

  0%|          | 0/7165 [00:00<?, ?it/s]

text column max_len: 822


In [12]:
input_col = "input"
sep = CFG.tokenizer.sep_token
train[input_col] = (
                    train["prompt_title"] + sep 
#                    + train_df["prompt_text"] + sep 
                    + train["prompt_question"] + sep 
                    + train["text"]
                  )

# input max Length

In [13]:
# ====================================================
# Define max_len
# ====================================================
lengths = []
tk0 = tqdm(train['input'].fillna("").values, total=len(train))
for text in tk0:
    length = len(CFG.tokenizer(text, add_special_tokens=False)['input_ids'])
    lengths.append(length)
CFG.max_len = max(lengths) + 2 # cls & sep
LOGGER.info(f"input column max_len: {CFG.max_len}")

  0%|          | 0/7165 [00:00<?, ?it/s]

input column max_len: 858


In [14]:
class FeedBackDataset(Dataset):
    def __init__(self, df, tokenizer, max_length):
        self.df = df
        self.max_len = CFG.max_len
        self.text = df['input'].values
        self.tokenizer = CFG.tokenizer
        self.targets = df[CFG.targets].values

    def __len__(self):
        return len(self.df)

    # staticmethod に書き換えたい
    def cut_head_and_tail(self, text):
        input_ids = self.tokenizer.encode(text)
        n_token = len(input_ids)

        if n_token == self.max_len:
            input_ids = input_ids
            attention_mask = [1 for _ in range(self.max_len)]
            token_type_ids = [1 for _ in range(self.max_len)]
        elif n_token < self.max_len:
            pad = [1 for _ in range(self.max_len-n_token)]
            input_ids = input_ids + pad
            attention_mask = [1 if n_token > i else 0 for i in range(self.max_len)]
            token_type_ids = [1 if n_token > i else 0 for i in range(self.max_len)]
        else:
            harf_len = (self.max_len-2)//2#256
            
            _input_ids = input_ids[1:-1]
            input_ids = [0]+ _input_ids[:harf_len] + _input_ids[-harf_len:] + [2]#取前256個跟後256個
            attention_mask = [1 for _ in range(self.max_len)]
            token_type_ids = [1 for _ in range(self.max_len)]

            if len(input_ids) < self.max_len:
                diff = self.max_len - len(input_ids)
                input_ids = [0]+ _input_ids[:harf_len] + _input_ids[-(harf_len+diff):] + [2]

        d = {
            "input_ids": input_ids,
            "attention_mask": attention_mask,
            "token_type_ids": token_type_ids,
        }
        return d

    def __getitem__(self, index):
        text = self.text[index]
        inputs = self.cut_head_and_tail(text)
        return {
            'input_ids':inputs['input_ids'],
            'attention_mask':inputs['attention_mask'],
            'target':self.targets[index]
            }

In [15]:
class Collate:
    def __init__(self, tokenizer, isTrain=True):
        self.tokenizer = tokenizer
        self.isTrain = isTrain

    def __call__(self, batch):
        
        output = dict()
        output["input_ids"] = [sample["input_ids"] for sample in batch]
        output["attention_mask"] = [sample["attention_mask"] for sample in batch]
        if self.isTrain:
            output["target"] = [sample["target"] for sample in batch]

        # calculate max token length of this batch
        batch_max = max([len(ids) for ids in output["input_ids"]])
        
        # add padding
        if self.tokenizer.padding_side == "right":#沒意義
            output["input_ids"] = [s + (batch_max - len(s)) * [self.tokenizer.pad_token_id] for s in output["input_ids"]]
            output["attention_mask"] = [s + (batch_max - len(s)) * [0] for s in output["attention_mask"]]
        else:
            output["input_ids"] = [(batch_max - len(s)) * [self.tokenizer.pad_token_id] + s for s in output["input_ids"]]
            output["attention_mask"] = [(batch_max - len(s)) * [0] + s for s in output["attention_mask"]]

        # convert to tensors
        output["input_ids"] = torch.tensor(output["input_ids"], dtype=torch.long)
        output["attention_mask"] = torch.tensor(output["attention_mask"], dtype=torch.long)
        if self.isTrain:
            output["target"] = torch.tensor(output["target"], dtype=torch.float)

        
        return output

In [16]:
collate_fn = Collate(CFG.tokenizer, isTrain=True)

In [17]:
def freeze(module):
    """
    Freezes module's parameters.
    """

    for parameter in module.parameters():
        parameter.requires_grad = False

def get_scheduler(cfg, optimizer, num_train_steps):
    if cfg.scheduler == 'linear':
        scheduler = get_linear_schedule_with_warmup(
            optimizer, num_warmup_steps=cfg.num_warmup_steps, num_training_steps=num_train_steps
        )
    elif cfg.scheduler == 'cosine':
        scheduler = get_cosine_schedule_with_warmup(
            optimizer, num_warmup_steps=cfg.num_warmup_steps, num_training_steps=num_train_steps, num_cycles=cfg.num_cycles
        )
    return scheduler


In [18]:
class FeedBackModel(nn.Module):
    def __init__(self, model_name):
        super(FeedBackModel, self).__init__()

        self.cfg = CFG
        self.config = AutoConfig.from_pretrained(model_name)
        self.config.hidden_dropout_prob = 0
        self.config.attention_probs_dropout_prob = 0

        self.model = AutoModel.from_pretrained(model_name, config=self.config)

        self.output = nn.Sequential(
            nn.LayerNorm(self.config.hidden_size),
            nn.Linear(self.config.hidden_size, self.cfg.target_size)
        )

        
        # Freeze
        if self.cfg.freezing:
            freeze(self.model.embeddings)
            freeze(self.model.encoder.layer[:2])

        # Gradient Checkpointing
        #if self.cfg.gradient_checkpoint:
        #    self.model.gradient_checkpointing_enable() 

        #if self.cfg.reinit_layers > 0:
        #    layers = self.model.encoder.layer[-self.cfg.reinit_layers:]
        #    for layer in layers:
        #        for module in layer.modules():
        #            self._init_weights(module)

    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            module.weight.data.normal_(mean=0.0, std=self.config.initializer_range)
            if module.bias is not None:
                module.bias.data.zero_()
        elif isinstance(module, nn.Embedding):
            module.weight.data.normal_(mean=0.0, std=self.config.initializer_range)
            if module.padding_idx is not None:
                module.weight.data[module.padding_idx].zero_()
        elif isinstance(module, nn.LayerNorm):
            module.bias.data.zero_()
            module.weight.data.fill_(1.0)

    def forward(self, ids, mask, token_type_ids=None, targets=None):
        if token_type_ids:
            transformer_out = self.model(ids, mask, token_type_ids)
        else:
            transformer_out = self.model(ids, mask)

        # simple CLS
        sequence_output = transformer_out[0][:, 0, :]

        logits = self.output(sequence_output)

        return logits

In [19]:
def criterion(outputs, targets):
    loss_fct = nn.MSELoss()
    loss = loss_fct(outputs, targets)
    return loss

def get_score(outputs, targets):
    mcrmse = []
    for i in range(CFG.target_size):
        mcrmse.append(
            metrics.mean_squared_error(
                targets[:, i],
                outputs[:, i],
                squared=False,
            ),
        )
    mcrmse = np.mean(mcrmse)
    return mcrmse
def get_result(oof_df):
    labels = oof_df[CFG.targets].values
    preds = oof_df[['pred_0', 'pred_1']].values
    score = get_score(preds, labels)
    LOGGER.info(f'Score: {score:<.4f}')

# Training And Validation Per Epoch

In [20]:
def train_one_epoch(model, optimizer, scheduler, dataloader, valid_loader, device, epoch, best_score, valid_labels):
    model.train()
    scaler = GradScaler(enabled=CFG.apex)

    dataset_size = 0
    running_loss = 0

    start = end = time.time()

    for step, data in enumerate(dataloader):
        
        ids = data['input_ids'].to(device, dtype=torch.long)
        mask = data['attention_mask'].to(device, dtype=torch.long)
        targets = data['target'].to(device, dtype=torch.float)

        batch_size = ids.size(0)

        with autocast(enabled=CFG.apex):
            outputs = model(ids, mask)
            loss = criterion(outputs, targets)

        #accumulate
        loss = loss / CFG.n_accumulate
        scaler.scale(loss).backward()
        if (step +1) % CFG.n_accumulate == 0:#n_accumulate=1
            # torch.nn.utils.clip_grad_norm_(model.parameters(), CFG.max_norm)
            scaler.step(optimizer)
            scaler.update()
            optimizer.zero_grad()
            if scheduler is not None:
                scheduler.step()
        running_loss += (loss.item() * batch_size)
        dataset_size += batch_size

        epoch_loss = running_loss / dataset_size

        end = time.time()

        if step % CFG.print_freq == 0 or step == (len(dataloader)-1):
            print('Epoch: [{0}][{1}/{2}] '
                  'Loss: [{3}]'
                  'Elapsed {remain:s} '
                  .format(epoch+1, step, len(dataloader), epoch_loss,
                          remain=timeSince(start, float(step+1)/len(dataloader))))

        if (step > 0) & (step % CFG.eval_freq == 0) :

            valid_epoch_loss, pred = valid_one_epoch(model, valid_loader, device, epoch)

            score = get_score(pred, valid_labels)

            LOGGER.info(f'Epoch {epoch+1} Step {step} - avg_train_loss: {epoch_loss:.4f}  avg_val_loss: {valid_epoch_loss:.4f}')
            LOGGER.info(f'Epoch {epoch+1} Step {step} - Score: {score:.4f}')

            if score < best_score:
                best_score = score
                LOGGER.info(f'Epoch {epoch+1} Step {step} - Save Best Score: {best_score:.4f} Model')
                torch.save({'model': model.state_dict(),
                            'predictions': pred},
                            OUTPUT_DIR+f"/model/{CFG.model.replace('/', '-')}_fold{fold}_best.pth")

            # model.train()

    gc.collect()

    return epoch_loss, valid_epoch_loss, pred, best_score

@torch.no_grad()
def valid_one_epoch(model, dataloader, device, epoch):
    model.eval()

    dataset_size = 0
    running_loss = 0

    start = end = time.time()
    pred = []

    for step, data in enumerate(dataloader):
        ids = data['input_ids'].to(device, dtype=torch.long)
        mask = data['attention_mask'].to(device, dtype=torch.long)
        targets = data['target'].to(device, dtype=torch.float)

        batch_size = ids.size(0)
        outputs = model(ids, mask)
        loss = criterion(outputs, targets)
        pred.append(outputs.to('cpu').numpy())

        running_loss += (loss.item()* batch_size)
        dataset_size += batch_size

        epoch_loss = running_loss / dataset_size

        end = time.time()

        if step % CFG.print_freq == 0 or step == (len(dataloader)-1):
            print('EVAL: [{0}/{1}] '
                  'Elapsed {remain:s} '
                  .format(step, len(dataloader),
                          remain=timeSince(start, float(step+1)/len(dataloader))))

    pred = np.concatenate(pred)
    return epoch_loss, pred

# Training And Validation Loop

In [21]:
def train_loop(fold):
    LOGGER.info(f'-------------fold:{fold} training-------------')

    train_data = train[train.fold != fold].reset_index(drop=True)
    valid_data = train[train.fold == fold].reset_index(drop=True)
    valid_labels = valid_data[CFG.targets].values

    trainDataset = FeedBackDataset(train_data, CFG.tokenizer, CFG.max_len)
    validDataset = FeedBackDataset(valid_data, CFG.tokenizer, CFG.max_len)
    
    

    train_loader = DataLoader(trainDataset,
                              batch_size = CFG.batch_size,
                              shuffle=True,
                              collate_fn = collate_fn,
                              num_workers = CFG.num_workers,
                              pin_memory = True,
                              drop_last=True)

    
    valid_loader = DataLoader(validDataset,
                              batch_size = CFG.batch_size * 2,
                              shuffle=False,
                              collate_fn = collate_fn,
                              num_workers = CFG.num_workers,
                              pin_memory = True,
                              drop_last=False)

    model = FeedBackModel(CFG.model)
    torch.save(model.config, OUTPUT_DIR+'/model/config.pth')
    model.to(device)
    optimizer = AdamW(model.parameters(), lr=CFG.lr, weight_decay=CFG.weigth_decay)
    num_train_steps = int(len(train_data) / CFG.batch_size * CFG.epochs)
    scheduler = get_scheduler(CFG, optimizer, num_train_steps)

    # loop
    best_score = 100

    for epoch in range(CFG.epochs):
        # if epoch == (CFG.epochs - 1):
        #     break

        start_time = time.time()

        train_epoch_loss, valid_epoch_loss, pred, best_score = train_one_epoch(model, optimizer, scheduler, train_loader, valid_loader, device, epoch, best_score, valid_labels)

        score = get_score(pred, valid_labels)

        elapsed = time.time() - start_time

        LOGGER.info(f'Epoch {epoch+1} - avg_train_loss: {train_epoch_loss:.4f}  avg_val_loss: {valid_epoch_loss:.4f}  time: {elapsed:.0f}s')
        LOGGER.info(f'Epoch {epoch+1} - Score: {score:.4f}')

        if score < best_score:
            best_score = score
            LOGGER.info(f'Epoch {epoch+1} - Save Best Score: {best_score:.4f} Model')
            torch.save({'model': model.state_dict(),
                        'predictions': pred},
                        OUTPUT_DIR+f"/model/{CFG.model.replace('/', '-')}_fold{fold}_best.pth")

    predictions = torch.load(OUTPUT_DIR+f"/model/{CFG.model.replace('/', '-')}_fold{fold}_best.pth",
                             map_location=torch.device('cpu'))['predictions']
    valid_data['pred_0'] = predictions[:, 0]
    valid_data['pred_1'] = predictions[:, 1]
    # valid_data['pred_2'] = predictions[:, 2]
    # valid_data['pred_3'] = predictions[:, 3]
    # valid_data['pred_4'] = predictions[:, 4]
    # valid_data['pred_5'] = predictions[:, 5]

    torch.cuda.empty_cache()
    gc.collect()

    return valid_data

In [22]:
if CFG.train:
    oof_df = pd.DataFrame()
    for fold in range(CFG.n_fold):
        if fold in CFG.trn_fold:
            _oof_df = train_loop(fold)
            oof_df = pd.concat([oof_df, _oof_df])
            LOGGER.info(f"========== fold: {fold} result ==========")
            get_result(_oof_df)
    oof_df = oof_df.reset_index(drop=True)
    LOGGER.info(f"========== CV ==========")
    get_result(oof_df)
    oof_df.to_csv(OUTPUT_DIR+f'_oof_df.csv', index=False)

-------------fold:0 training-------------
Some weights of the model checkpoint at microsoft/deberta-v3-base were not used when initializing DebertaV2Model: ['mask_predictions.classifier.bias', 'lm_predictions.lm_head.dense.weight', 'mask_predictions.LayerNorm.weight', 'mask_predictions.LayerNorm.bias', 'lm_predictions.lm_head.bias', 'mask_predictions.classifier.weight', 'mask_predictions.dense.weight', 'lm_predictions.lm_head.LayerNorm.bias', 'mask_predictions.dense.bias', 'lm_predictions.lm_head.LayerNorm.weight', 'lm_predictions.lm_head.dense.bias']
- This IS expected if you are initializing DebertaV2Model from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaV2Model from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceCl

Epoch: [1][0/5108] Loss: [2.671222686767578]Elapsed 0m 1s (remain 142m 16s) 
Epoch: [1][100/5108] Loss: [1.0035157612142254]Elapsed 0m 10s (remain 8m 30s) 
Epoch: [1][200/5108] Loss: [0.9806683814855042]Elapsed 0m 18s (remain 7m 42s) 
Epoch: [1][300/5108] Loss: [0.8687996745080164]Elapsed 0m 27s (remain 7m 20s) 
Epoch: [1][400/5108] Loss: [0.7793878867857604]Elapsed 0m 36s (remain 7m 5s) 
Epoch: [1][500/5108] Loss: [0.7169835375895642]Elapsed 0m 44s (remain 6m 52s) 
Epoch: [1][600/5108] Loss: [0.6722009869712181]Elapsed 0m 53s (remain 6m 41s) 
Epoch: [1][700/5108] Loss: [0.636118996496992]Elapsed 1m 2s (remain 6m 30s) 
Epoch: [1][800/5108] Loss: [0.599341884369154]Elapsed 1m 10s (remain 6m 20s) 
Epoch: [1][900/5108] Loss: [0.5793823420322097]Elapsed 1m 19s (remain 6m 12s) 
Epoch: [1][1000/5108] Loss: [0.5566850868251007]Elapsed 1m 29s (remain 6m 6s) 
Epoch: [1][1100/5108] Loss: [0.5421935644879673]Elapsed 1m 38s (remain 5m 59s) 
Epoch: [1][1200/5108] Loss: [0.531760514137139]Elapsed 1m

Epoch 1 Step 1560 - avg_train_loss: 0.4893  avg_val_loss: 0.3078
Epoch 1 Step 1560 - Score: 0.5500
Epoch 1 Step 1560 - Save Best Score: 0.5500 Model


EVAL: [1028/1029] Elapsed 1m 48s (remain 0m 0s) 
Epoch: [1][1600/5108] Loss: [0.48678002259118325]Elapsed 4m 19s (remain 9m 29s) 
Epoch: [1][1700/5108] Loss: [0.4762185636668466]Elapsed 4m 29s (remain 8m 59s) 
Epoch: [1][1800/5108] Loss: [0.46885177440245523]Elapsed 4m 38s (remain 8m 31s) 
Epoch: [1][1900/5108] Loss: [0.45924669681219343]Elapsed 4m 48s (remain 8m 6s) 
Epoch: [1][2000/5108] Loss: [0.44988548614957874]Elapsed 4m 57s (remain 7m 42s) 
Epoch: [1][2100/5108] Loss: [0.4440156893883529]Elapsed 5m 7s (remain 7m 19s) 
Epoch: [1][2200/5108] Loss: [0.4373977525700866]Elapsed 5m 16s (remain 6m 58s) 
Epoch: [1][2300/5108] Loss: [0.4294021329410401]Elapsed 5m 26s (remain 6m 37s) 
Epoch: [1][2400/5108] Loss: [0.42101600636578]Elapsed 5m 35s (remain 6m 18s) 
Epoch: [1][2500/5108] Loss: [0.41762667971920503]Elapsed 5m 45s (remain 5m 59s) 
Epoch: [1][2600/5108] Loss: [0.4093976663247953]Elapsed 5m 54s (remain 5m 41s) 
Epoch: [1][2700/5108] Loss: [0.4053047892166705]Elapsed 6m 4s (remain 

Epoch 1 Step 3120 - avg_train_loss: 0.3911  avg_val_loss: 0.2587
Epoch 1 Step 3120 - Score: 0.5064
Epoch 1 Step 3120 - Save Best Score: 0.5064 Model


...

Epoch 4 Step 4680 - avg_train_loss: 0.0927  avg_val_loss: 0.4309
Epoch 4 Step 4680 - Score: 0.6562


EVAL: [551/552] Elapsed 0m 58s (remain 0m 0s) 
Epoch: [4][4700/6062] Loss: [0.09259802100141602]Elapsed 10m 20s (remain 2m 59s) 
Epoch: [4][4800/6062] Loss: [0.09277069952296692]Elapsed 10m 30s (remain 2m 45s) 
Epoch: [4][4900/6062] Loss: [0.0928406447903042]Elapsed 10m 39s (remain 2m 31s) 
Epoch: [4][5000/6062] Loss: [0.09275733919158025]Elapsed 10m 49s (remain 2m 17s) 
Epoch: [4][5100/6062] Loss: [0.09285939716493964]Elapsed 10m 58s (remain 2m 4s) 
Epoch: [4][5200/6062] Loss: [0.09261499285555504]Elapsed 11m 8s (remain 1m 50s) 
Epoch: [4][5300/6062] Loss: [0.09260875183244074]Elapsed 11m 17s (remain 1m 37s) 
Epoch: [4][5400/6062] Loss: [0.09285678580976171]Elapsed 11m 27s (remain 1m 24s) 
Epoch: [4][5500/6062] Loss: [0.09292557010090899]Elapsed 11m 36s (remain 1m 11s) 
Epoch: [4][5600/6062] Loss: [0.09320859423792013]Elapsed 11m 46s (remain 0m 58s) 
Epoch: [4][5700/6062] Loss: [0.09320084460013278]Elapsed 11m 55s (remain 0m 45s) 
Epoch: [4][5800/6062] Loss: [0.09327824713774432]Elaps

Epoch 4 - avg_train_loss: 0.0934  avg_val_loss: 0.4309  time: 750s
Epoch 4 - Score: 0.6562


Epoch: [4][6061/6062] Loss: [0.09339652683837883]Elapsed 12m 29s (remain 0m 0s) 


========== fold: 3 result ==========
Score: 0.6056
========== CV ==========
Score: 0.5579
